In [1]:
import tensorflow as tf
from src.config import ROOT_DIR, IMG_SHAPE
import numpy as np
import pandas as pd
import visualkeras

In [2]:
ds = pd.read_csv('fer2013.csv')
ds.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [3]:
ds['emotion'] = ds['emotion'].transform(lambda x: {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Neutral', 5: 'Sad', 6: 'Surprise'}[x])


In [3]:
import plotly.express as px

# fig = px.histogram(ds, x='emotion', color='emotion', title='Histogram zbioru danych ze względu na emocje', labels={'emotion': 'Emocje'}, text_auto=True)
# fig.update_layout(bargap=0.2, xaxis_title='Emocje', yaxis_title='Liczność')
fig = px.histogram(x=['Angry', 'Fear', 'Neutral', 'Surprise', 'Happy', 'Sad', 'Disgust'], y=[8989,8989,8989,8989,8989,8989,8989,], title='Histogram zbioru danych po augmentacji')
fig.update_layout(bargap=0.2, xaxis_title='Emocje', yaxis_title='Liczność')
fig.write_image('augmented_ds.png')

In [6]:
print(tf.config.list_physical_devices('GPU'))

BATCH_SIZE = 512
EPOCHS = 50

TRAINNG_DIR = ROOT_DIR / "../../raw/images" / "Training"
VAL_DIR = ROOT_DIR / "../../raw/images" / "PublicTest"
print(TRAINNG_DIR)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
/Users/wiktorpieklik/studies/2sem/sieci_neuronowe/projekt/src/notebooks/../../raw/images/Training


In [ ]:
training_ds = tf.keras.utils.image_dataset_from_directory(
    directory=TRAINNG_DIR,
    label_mode='categorical',
    batch_size=None,
    color_mode='grayscale',
    image_size=IMG_SHAPE,
    shuffle=True
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory=VAL_DIR,
    label_mode='categorical',
    batch_size=None,
    color_mode='grayscale',
    image_size=IMG_SHAPE
)
rescale = tf.keras.layers.Rescaling(1. / 255)

training_ds = training_ds.batch(BATCH_SIZE).map(lambda img, label: (rescale(img), label)).shuffle(30000).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).map(lambda img, label: (rescale(img), label)).prefetch(tf.data.AUTOTUNE)


In [8]:
weights = {0: 0, 1: 0, 2: 0}
mapper = {
    'Happy': 0,
    'Neutral': 1,
    'Sad': 2
}
labels = ['Happy', 'Neutral', 'Sad']
path = ROOT_DIR / "../../raw/images" / "Training"
total = len([obj for obj in path.rglob('*') if not obj.is_dir()])
for i, label in enumerate(labels):
    weights[i] = (1 / len(list((path / label).rglob("*")))) * (total / 3.0)

weights


57683

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(IMG_SHAPE + (1,)),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(7,7), padding='same', activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(7,7), padding='same', activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(5,5), padding='same', activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(5,5), padding='same', activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(5,5), padding='same', activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GlobalAvgPool2D(),
    tf.keras.layers.Dense(3, activation="softmax")
])

visualkeras.layered_view(model, legend=True, to_file='custom1.png')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 48, 48, 32)        320       
                                                                 
 batch_normalization_24 (Bat  (None, 48, 48, 32)       128       
 chNormalization)                                                
                                                                 
 dropout_6 (Dropout)         (None, 48, 48, 32)        0         
                                                                 
 conv2d_25 (Conv2D)          (None, 48, 48, 32)        9248      
                                                                 
 batch_normalization_25 (Bat  (None, 48, 48, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 24, 24, 32)      

In [ ]:
from livelossplot import PlotLossesKeras

def scheduler(epoch, lr):
    if epoch < 20:
        return lr
    return lr * np.exp(-.15)

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(monitor='val_loss', save_best_only=True, filepath='best_model.h5'),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1),
    tf.keras.callbacks.LearningRateScheduler(scheduler),
    PlotLossesKeras()
]

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=.01),
    metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.AUC()]
  )

In [ ]:
history = model.fit(
    training_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=callbacks,
    # class_weight=weights
)

In [ ]:
model.save('best.h5')

In [ ]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    directory=ROOT_DIR / "../../raw/images" / "PrivateTest",
    label_mode='categorical',
    batch_size=None,
    color_mode='grayscale',
    image_size=IMG_SHAPE
)

test_ds = test_ds.batch(BATCH_SIZE).map(lambda img, label: (rescale(img), label))

In [ ]:
model.evaluate(test_ds)